In [37]:
# Author: iTrue
# Date: 2023-04-14
# Description: Scan WiFi SSIDs

import os
import pandas as pd
import subprocess

def get_wifi_networks():
    cmd = 'netsh wlan show networks mode=Bssid'
    output = subprocess.check_output(cmd, shell=True)
    output = output.decode('utf-8')
    networks = []
    network = {}
    for line in output.split('\n'):
        line = line.strip()
        if line.startswith('SSID'):
            if network:
                networks.append(network)
            network = {}
            ssid = line.split(':')[1].strip()
            network['wifiname'] = ssid
        elif line.startswith('BSSID'):
            bssid = line.split(':')[1].strip()
            network['bssid'] = bssid
        elif line.startswith('Authentication'):
            auth = line.split(':')[1].strip()
            network['authentication'] = auth
    if network:
        networks.append(network)
    df = pd.DataFrame(networks)
    return df

def scan_wifi():
    if os.name == 'nt':
        return get_wifi_networks()

    else:
        import network

        wlan = network.WLAN(network.STA_IF)
        wlan.active(True)
        nets = wlan.scan()
        ssids = [n[0].decode() for n in nets]
        return ssids
    
if __name__ == '__main__':
    # import pandas as pd
    # # Create an empty DataFrame
    # df = pd.DataFrame(columns=['wifi name', 'ssid'])
    # ssids = scan_wifi()

    # # Create a list of dictionaries for appending data
    # data = [{'wifi name': ssid[0], 'ssid': ssid[1]} for ssid in ssids]

    # # Concatenate the list of dictionaries to the DataFrame
    # df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)

    # # Print the updated DataFrame
    # print(df)
    print(scan_wifi())


    wifiname authentication bssid
0  BTMSecure  WPA2-Personal    80


In [ ]:
# Author: iTrue
# Date: 2023-04-14
# Description: Connect to WiFi with Python known SSID and password

import network

def connect_to_wifi(ssid, password):
    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    if not wlan.isconnected():
        print('connecting to network...')
        wlan.connect(ssid, password)
        while not wlan.isconnected():
            pass
    print('network config:', wlan.ifconfig())